## Setup

In [1]:
#Importing required packages
from __future__ import absolute_import, division, print_function

import tensorflow as tf
tf.enable_eager_execution()

import numpy as np
import os
import time
import random

In [6]:
#Reading in previously scraped transcripts
text = ''
for file in os.listdir(os.getcwd()):
    if os.path.isfile(file) and file != "Final Training and Text Generation.ipynb":
        print(file)
        text += open(file, 'rb').read().decode(encoding='utf-8')
    
print ('Length of text: {} characters'.format(len(text)))
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

#70 Stolen Valor | Reply All
#90 Matt Lieber Goes to Dinner | Reply All
#14 The Art of Making and Fixing Mistakes | Reply All
#142 We Didn't Start The Fire | Reply All
#82 Hello? | Reply All
#122 The QAnon Code ⚡️⚡️ | Reply All
#22 BONUS: The Man Who Refused To Email | Reply All
#141 Adam Pisces and the $2 Coke | Reply All
#110 The Antifa Supersoldier Spectacular | Reply All
#128 The Crime Machine, Part II | Reply All
#65 On the Inside, Part II | Reply All
#29 The Takeover | Reply All
#127 The Crime Machine, Part I | Reply All
#93 Beware All | Reply All
#104 The Case of the Phantom Caller | Reply All
#130 The Snapchat Thief | Reply All
#94 Obfuscation | Reply All
#92 Favor Atender: The Return | Reply All
#6 This Proves Everything + The Best Hold Music in the World | Reply All
#20 I Want To Break Free | Reply All
#67 On the Inside, Part IV | Reply All
#55 The Line | Reply All
#30 The Man In The FBI Hat | Reply All
#120 INVCEL | Reply All
#84 Past, Present, Future 2 | Reply All
#135 Robo

In [7]:
#Vectorizing text
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
#Creating training examples
seq_length = 107
examples_per_epoch = len(text)//seq_length

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


In [9]:
#Method to duplicate and shift to form input and target
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [10]:
#Creating training batches
BATCH_SIZE = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## Build the Model

In [11]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension 
embedding_dim = 256
# Number of RNN units
rnn_units = 1024

if tf.test.is_gpu_available():
    rnn = tf.keras.layers.CuDNNGRU
else:
    import functools
    rnn = functools.partial(tf.keras.layers.GRU, recurrent_activation='sigmoid')

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[batch_size, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

In [13]:
model = build_model(
  vocab_size = len(vocab), 
  embedding_dim=embedding_dim, 
  rnn_units=rnn_units, 
  batch_size=BATCH_SIZE)

## Train the Model

In [14]:
def loss(labels, logits):
#   return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
  return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)

model.compile(optimizer = tf.train.AdamOptimizer(), loss = loss)

In [15]:
# Directory where the checkpoints will be saved
checkpoint_dir = './text50'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

In [28]:
EPOCHS=50

In [29]:
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

Epoch 1/50
1305/1305 [==============================] - 94s 72ms/step - loss: 1.6828
Epoch 2/50
1305/1305 [==============================] - 89s 68ms/step - loss: 1.2289
Epoch 3/50
1305/1305 [==============================] - 87s 66ms/step - loss: 1.1512
Epoch 4/50
1305/1305 [==============================] - 95s 73ms/step - loss: 1.1070
Epoch 5/50
1305/1305 [==============================] - 87s 67ms/step - loss: 1.0772
Epoch 6/50
1305/1305 [==============================] - 82s 63ms/step - loss: 1.0571
Epoch 7/50
1305/1305 [==============================] - 85s 65ms/step - loss: 1.0439
Epoch 8/50
1305/1305 [==============================] - 82s 63ms/step - loss: 1.0369
Epoch 9/50
1305/1305 [==============================] - 88s 67ms/step - loss: 1.0337
Epoch 10/50
1305/1305 [==============================] - 84s 65ms/step - loss: 1.0359
Epoch 11/50
1305/1305 [==============================] - 81s 62ms/step - loss: 1.0402
Epoch 12/50
1305/1305 [==============================] - 89s 68

In [16]:
tf.train.latest_checkpoint(checkpoint_dir)

'./text50/ckpt_50'

In [17]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            29696     
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 116)            118900    
Total params: 4,086,900
Trainable params: 4,086,900
Non-trainable params: 0
_________________________________________________________________


## Generate Text

In [18]:
def generate_text(num_generate, temp, model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  #  num_generate = 65000

  # Converting our start string to numbers (vectorizing) 
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
    text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
    temperature = temp

  # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
      # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()
      
      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)
      
        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
file = open("MUSIC65_100","a")
file.write(generate_text(65000, model, start_string=u"[THEME MUSIC]"))
file.close()

In [33]:
file = open("Convo.txt","a")
i=0
text  = "ALEX GOLDMAN"
while i < 25:
    text = generate_text(random.randint(30,108), 1.0, model, start_string=text)
    text = text + '\n'
    file.write(text)
    i+=1

file.close()

In [16]:
file = open("ConvoPJ.txt","a")
i=0
text  = "PJ VOGT:"
while i < 25:
    text = generate_text(random.randint(30,108), 0.5, model, start_string=text)
    text = text + '\n'
    file.write(text)
    i+=1

file.close()

In [17]:
file = open("ConvoMusic.txt","a")
i=0
text  = u"[THEME MUSIC]"
while i < 25:
    text = generate_text(random.randint(30,108), 1.5, model, start_string=text)
    text = text + '\n'
    file.write(text)
    i+=1

file.close()

In [19]:
file = open("Convo108.txt","a")
i=0
text  = u"[THEME MUSIC]"
while i < 25:
    text = generate_text(108, 1.0, model, start_string=text)
    text = text + '\n'
    file.write(text)
    i+=1

file.close()

In [20]:
file = open("ConvoHosts2.txt","a")
i=0
hosts  = [ u"ALEX GOLDMAN:",
          u"PJ VOGT:", u"Damiano Marchetti:",
          u"Phia Bennin:", u"Sruthi Pinnamaneni:",
          u"Anna Foley:", u"ALEX BLUMBERG:" ]
while i < 25:
    text = generate_text(108, 1.0, model, start_string=random.choice(hosts))
    text = text + '\n' + '\n'
    file.write(text)
    i+=1

file.close()